In [ ]:
import pandas as pd
import numpy as np
import dsar
from itertools import product
pd.set_option('display.max_columns', None)

In [ ]:
basic_col = ['app_age',
             'sex_male',
#             'ins_sex',
            'occ_wp_pb',
            'occ_tpd',
            'occ_adb',
            'occ_add',
            'occ_cab_cabp_iab',
            'occ_mdi',
            'g_height_cm',
            'g_weight_kg',
             'diff_low',
            'diff_high',
            'bmi_clip',
            ]

smoke_alcohol_col = ['smoker',
                    'g_smoke_daily_pcs',
                    'g_alcohol',
                    'g_alcohol_daily_ml',
                    ]

generic_med_col = [
'g_self_cancer',
'g_self_digestive_condition',
'g_self_endocrine_disorder',
'g_self_heart_condition',
'g_self_mental_condition',
'g_self_muscular_disorder',
'g_self_nervous_system_disorder',
'g_self_reproductive_disorder',
'g_self_respiratory_disorder',
'g_self_urinary_tract_disorder']

cancer_sub_col = ['g_self_cancer_tumor']

digest_sub_col = ['g_self_digestive_hepatitis',
'g_self_digestive_other_gastrointestinal_disorder',
'g_self_digestive_other_liver_disorder'
]

respiratory_sub_col = ['g_self_respiratory_asthma' ,
'g_self_respiratory_other_respiratory_disorder']

family_med_col = ['g_fam_cancer',
                    'g_fam_digestive_condition',
                    'g_fam_endocrine_disorder',
                    'g_fam_other_disorder']

cols = basic_col+smoke_alcohol_col + generic_med_col + cancer_sub_col + digest_sub_col+ respiratory_sub_col + family_med_col
# cols = ', '.join(cols) if cols else '*'

col_rename = {col: 'v_'+ col for col in cols}

In [ ]:
uw_sim_models = pd.DataFrame(list(product([0,1],repeat=len(generic_med_col))), columns=generic_med_col)
uw_sim_models = uw_sim_models[uw_sim_models.sum(axis=1)>=len(generic_med_col)-3]

In [ ]:
for col in cancer_sub_col:
    uw_sim_models[col] = uw_sim_models.g_self_cancer
for col in digest_sub_col:
    uw_sim_models[col] = uw_sim_models.g_self_digestive_condition
for col in respiratory_sub_col:
    uw_sim_models[col] = uw_sim_models.g_self_respiratory_disorder
uw_sim_models = uw_sim_models.reset_index(drop=True)
uw_sim_models[family_med_col] = 0
uw_sim_models[basic_col + smoke_alcohol_col] = 1
uw_sim_models.rename(columns=col_rename, inplace=True)
full_feat = pd.DataFrame(np.ones_like(uw_sim_models.columns).reshape(1,-1), columns=uw_sim_models.columns)
uw_sim_models = pd.concat([full_feat,uw_sim_models], axis=0).reset_index(drop=True)

In [ ]:
len(col)

In [ ]:
uw_sim_models

In [ ]:
# with dsar.psql_con('HB') as con:
#     sql = f"""select {columns} from hb.hb_poc_premiums_rv3
#                             where plan_region='HK' and vhis_compatible=1 and year_seq<=5
#                             ORDER BY contract_no, year_seq
#                             limit 100
#                                 """
#     df_sample = pd.read_sql(sql=sql, con=con)

    

In [ ]:
# cols = ['g_alcohol',
# 'g_self_digestive_condition',
# 'g_self_endocrine_disorder',
# 'g_self_heart_condition',
# 'g_self_muscular_disorder',
# 'g_self_other_disorder',
# 'g_self_respiratory_disorder',
# 'g_self_urinary_tract_disorder',
# 'g_self_cancer_tumor',
# 'g_self_digestive_hepatitis',
# 'g_self_digestive_other_gastrointestinal_disorder',
# 'g_self_digestive_other_liver_disorder',
# 'g_self_respiratory_asthma',
# 'g_self_respiratory_other_respiratory_disorder',
# 'g_fam_cancer',
# 'g_fam_digestive_condition',
# 'g_fam_endocrine_disorder',
# 'g_fam_other_disorder',
# 'app_age',
# 'smoker',
# 'g_height_cm',
# 'g_weight_kg',
# 'g_smoke_daily_pcs',
# 'g_alcohol_daily_ml',
# 'occ_wp_pb',
# 'occ_tpd',
# 'occ_adb',
# 'occ_add',
# 'occ_cab_cabp_iab',
# 'occ_mdi',
# 'sex_male',
# 'bmi',
# 'clean_record']

# cols = ['v_'+ col for col in cols]

In [ ]:
# uw_sim_models = pd.DataFrame.from_records(data = np.ones_like(cols).reshape(1,-1), columns = cols)
uw_sim_models[['fit_status','tune_status']]='W'
uw_sim_models[['fit_s3_loc','fit_last_update_time', 'fit_start_time', 'fitting_machine',
               'tune_last_update_time','tuning_machine']]= ""
uw_sim_models.insert(0, 'model_id', uw_sim_models.index)

In [ ]:
uw_sim_preds = pd.DataFrame(data = {"model_id": int(), "contract_no": str(), "pred": float(),
                                  "train": int()}, index=[])

In [ ]:
uw_sim_preds

In [ ]:
uw_sim_models.dtypes

In [ ]:
# with dsar.psql_con('HB') as con:
#     uw_sim_preds.to_sql(name=f'uw_sim_preds_v1', con=con, schema='hb',index=False, if_exists='replace')

In [ ]:
out_col = ['model_id', 'pref_percentile', 'std_percentile', 'pref_prop', 'std_prop','substd_prop',
           'pref_loss', 'std_loss',
           'substd_loss','S_P_loss','agreement_rate', 'nstd_agreement_rate',
        'pref_decline_prop', 'std_decline_prop', 'substd_decline_prop',
        'pref_size', 'std_size','substd_size',
       'pref_decline', 'pref_exclusion', 'pref_loading',
      'std_decline', 'std_exclusion', 'std_loading',
    'substd_decline', 'substd_exclusion', 'substd_loading',
    'pref_std', 'std_std', 'substd_std'
]
int_col = ['model_id', 'pref_percentile', 'std_percentile','pref_size', 'std_size','substd_size',
       'pref_decline', 'pref_exclusion', 'pref_loading',
      'std_decline', 'std_exclusion', 'std_loading',
    'substd_decline', 'substd_exclusion', 'substd_loading',
    'pref_std', 'std_std', 'substd_std']
uw_sim_tune = pd.DataFrame(columns = out_col)
uw_sim_tune = uw_sim_tune.astype(float)
uw_sim_tune[int_col] = uw_sim_tune[int_col].astype(int)

In [ ]:
uw_sim_tune

In [ ]:
# with dsar.psql_con('HB') as con:
#     uw_sim_tune.to_sql(name=f'uw_sim_tune', con=con, schema='hb',index=False, if_exists='replace')

In [ ]:
# uw_sim_models.to_csv('queue_table_v2.csv', index=False)

In [ ]:
# uw_sim_models = pd.read_csv('./queue_table_v2.csv',)

In [ ]:
with dsar.psql_con('HB') as con:
    uw_sim_models = pd.read_sql('select * from hb.uw_sim_models', con=con)

In [ ]:
uw_sim_models[uw_sim_models.drop('model_id', axis=1).sum(axis=1)==47]

In [ ]:
uw_sim_models_append = uw_sim_models.drop(uw_sim_models.index, axis=0)

In [ ]:
# bootstrap
uw_sim_models_bootstrap = uw_sim_models[uw_sim_models.model_id.isin([178,229,233,236])]
uw_sim_models_bootstrap = uw_sim_models_bootstrap.rename(columns={'model_id': 'original_model_id'})
uw_sim_models_bootstrap['bootstrap_id'] = 0
uw_sim_models_bootstrap_temp = uw_sim_models_bootstrap.copy()
for i in range(1,20):
    uw_sim_models_bootstrap_temp['bootstrap_id'] = i
    uw_sim_models_bootstrap = pd.concat([uw_sim_models_bootstrap,uw_sim_models_bootstrap_temp], axis=0)
uw_sim_models_bootstrap = uw_sim_models_bootstrap.sort_values(by=['original_model_id', 'bootstrap_id']).reset_index(drop=True)
uw_sim_models_bootstrap = uw_sim_models_bootstrap.reset_index().rename(columns={'index':'model_id'})
uw_sim_models_bootstrap[['fit_status','tune_status', 'adj_tune_status']]='W'
uw_sim_models_bootstrap[['fit_s3_loc','fit_last_update_time', 'fit_start_time', 'fitting_machine',
               'tune_last_update_time','tuning_machine']]= ""
uw_sim_models_bootstrap

In [ ]:
# with dsar.psql_con('HB') as con:
#     uw_sim_models_bootstrap.to_sql(name=f'uw_sim_models_bootstrap', con=con, schema='hb',index=False, if_exists='append')

In [ ]:
# uw_sim_models_append[['v_' + col for col in generic_med_col]] = list(product([0,1],repeat=len(generic_med_col)))

In [ ]:
# uw_sim_models.query('model_id==150').to_csv('extra_condition.csv', index=False)

In [ ]:
# uw_sim_models_append = uw_sim_models_append[uw_sim_models_append.sum(axis=1)==2].reset_index(drop=True)
extra_condition = pd.read_csv('extra_condition.csv', dtype=uw_sim_models_append.dtypes.values)

In [ ]:
extra_condition

In [ ]:
uw_sim_models_append = pd.concat([extra_condition,uw_sim_models_append],axis=0, ignore_index=True)
uw_sim_models_append[['fit_status','tune_status']]='W'
uw_sim_models_append[['fit_s3_loc','fit_last_update_time', 'fit_start_time', 'fitting_machine',
               'tune_last_update_time','tuning_machine']]= ""
# uw_sim_models_append[['v_' + col for col in cancer_sub_col + digest_sub_col + respiratory_sub_col + family_med_col]] = 0
# uw_sim_models_append[['v_' + col for col in basic_col + smoke_alcohol_col]] = 1
uw_sim_models_append = uw_sim_models_append.reset_index(drop=True)
uw_sim_models_append['model_id'] = uw_sim_models_append.index + uw_sim_models.model_id.max() +1

In [ ]:
uw_sim_models_append

In [ ]:
# with dsar.psql_con('HB') as con:
#     uw_sim_models_append.to_sql(name=f'uw_sim_models', con=con, schema='hb',index=False, if_exists='append')

In [ ]:
# with dsar.psql_con('HB') as con:
# #     con.execute("""ALTER TABLE hb.uw_sim_models 
# #                 ADD adj_tune_status CHAR(1)""") 
#     con.execute("""UPDATE hb.uw_sim_models 
#                 SET adj_tune_status ='W' """)


In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""DELETE from hb.uw_sim_tune_adj
#          ;""")  

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""ALTER TABLE hb.uw_sim_models 
#                 ADD a_g_self_cancer INT, 
#                 ADD a_g_self_digestive_condition INT,
#                 ADD a_g_self_endocrine_disorder INT,
#                 ADD a_g_self_heart_condition INT,
#                 ADD a_g_self_mental_condition INT,
#                 ADD a_g_self_muscular_disorder INT,
#                 ADD a_g_self_nervous_system_disorder INT,
#                 ADD a_g_self_reproductive_disorder INT,
#                 ADD a_g_self_respiratory_disorder INT,
#                 ADD a_g_self_urinary_tract_disorder INT
#                 """)   


In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""UPDATE hb.uw_sim_models
#     SET 
#     a_g_self_cancer = 0,
#     a_g_self_digestive_condition = 0,
#     a_g_self_endocrine_disorder = 0,
#     a_g_self_heart_condition = 0,
#     a_g_self_mental_condition = 0,
#     a_g_self_muscular_disorder = 0,
#     a_g_self_nervous_system_disorder = 0,
#     a_g_self_reproductive_disorder = 0,
#     a_g_self_respiratory_disorder = 0,
#     a_g_self_urinary_tract_disorder = 0
#     """)

In [ ]:
# uw_sim_models[uw_sim_models[['v_g_self_cancer', 'v_g_self_digestive_condition',
#        'v_g_self_endocrine_disorder', 'v_g_self_heart_condition',
#        'v_g_self_mental_condition', 'v_g_self_muscular_disorder',
#        'v_g_self_nervous_system_disorder', 'v_g_self_reproductive_disorder',
#        'v_g_self_respiratory_disorder', 'v_g_self_urinary_tract_disorder',
#        'v_g_self_cancer_tumor', 'v_g_self_digestive_hepatitis']].sum(axis=1)==len(['v_g_self_cancer', 'v_g_self_digestive_condition',
#        'v_g_self_endocrine_disorder', 'v_g_self_heart_condition',
#        'v_g_self_mental_condition', 'v_g_self_muscular_disorder',
#        'v_g_self_nervous_system_disorder', 'v_g_self_reproductive_disorder',
#        'v_g_self_respiratory_disorder', 'v_g_self_urinary_tract_disorder',
#        'v_g_self_cancer_tumor', 'v_g_self_digestive_hepatitis'])].to_csv('extra_condition.csv')

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""ALTER TABLE hb.uw_sim_models
#         ADD PRIMARY KEY (model_id);""")   

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""ALTER TABLE hb.uw_sim_preds_v1
#         ADD UNIQUE (contract_no, model_id);""")   

In [ ]:
# with dsar.psql_con('HB') as con:
#     uw_sim_models.to_sql(name=f'uw_sim_models_test', con=con, schema='hb',index=False, if_exists='replace')

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""ALTER TABLE hb.uw_sim_models_test
#         ADD PRIMARY KEY (model_id);""")   

In [ ]:
# uw_sim_models[['fit_s3_loc','fit_last_update_time', 'fit_start_time', 'fitting_machine',
#                'tune_last_update_time','tune_last_update_time','tuning_machine']] = ""

In [ ]:
# uw_sim_models = uw_sim_models.head()

In [ ]:
# with dsar.psql_con('HB') as con:
#     uw_sim_models.to_sql(name=f'uw_sim_models', con=con, schema='hb',index=False, if_exists='replace')

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""UPDATE hb.uw_sim_models
#     SET fit_status = 'W', tune_status = 'W'
#     Where fit_status = 'D' and model_id>=229""")

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""DELETE from hb.uw_sim_models
#         where model_id <=175 and model_id>=1 ;""")  

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""DELETE from hb.uw_sim_tune
#         where model_id <=175 and model_id>=1 ;""")   

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""DELETE from hb.uw_sim_preds_v1
#         where model_id <=175 and model_id>=1 ;""")   

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""UPDATE hb.uw_sim_models
#     SET tune_status = 'W'
#     """)

In [ ]:
# with dsar.psql_con('HB') as con:
#     con.execute("""DROP TABLE hb.uw_sim_preds_bootstrap
#     """)

# with dsar.psql_con('HB') as con:
#     con.execute("""DROP TABLE hb.uw_sim_tune_adj_bootstrap
#     """)